In [150]:
import os, fnmatch
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd

In [151]:
import sys
import json
import glob


In [152]:

BASE_PATH = '/data/archive/downstream/'
SOURCES = ['TARGET', 'TCGA', 'THR', 'TH']
def donor_to_samples(donor_id):
    sample_paths = glob.glob(BASE_PATH + donor_id + '*')
    sample_ids = [p[p.rfind('/')+1:] for p in sample_paths]
    return sample_ids

In [153]:
targetIds = donor_to_samples('TARGET')
thoIds = donor_to_samples('TH0')
thrIds = donor_to_samples('THR')
tgcaIds = donor_to_samples('TCGA')


In [154]:
listOfPathsToUMEND = sorted(glob.glob("/data/archive/downstream/" + 'TH0*/secondary/ucsc_cgl*/RSEM/Hugo' + '/rsem_genes.hugo.results'))
listOfPathsToDisease = sorted(glob.glob('/data/archive/downstream/TH0*/tertiary/treehouse-[8]*[!_compv5]/diagnosed_disease.txt'))

In [170]:
clinical = pd.read_csv("/data/archive/compendium/v5/clinical.tsv",sep='\t')

ribodDiagnosis = pd.read_csv("riboDepleted_samples_that_passedQC_and_have_known_diagnosis.tsv", sep='\t')

ribodDiagnosis=ribodDiagnosis.rename(columns={'Treehouse SAMPLE identifier':'th_sampleid','Diagnosis/Disease':'disease'})

methods = pd.read_csv("TranscriptMethod_THPEDv1.csv")

methods = methods.rename(columns={'Treehouse SAMPLE identifier':'th_sampleid'})

clinicalIdTissue = clinical[['th_sampleid','anat_sample','disease']] 

In [365]:
df = pd.merge(clinicalIdTissue, ribodDiagnosis, how='outer', on='th_sampleid')
df = pd.merge(df,methods,how='left',on='th_sampleid')
df.disease_y.fillna(df.disease_x, inplace=True)
df=df.rename(columns={'disease_y':'disease','anat_sample':'tissue'})

del df['disease_x']

df.tissue.fillna('NA',inplace=True)

ribodGlioblastomaSet = df.loc[df['TR_method']=='RiboMinus'].loc[df['disease']=='glioblastoma multiforme']

polyaGliomaBrainSet = df.loc[df['disease']=="glioma"].loc[df['tissue']=="brain"]

compendium = pd.read_hdf("/data/archive/compendium/v5/v5_hugo_log2tpm.11340x58581.2018-02-03.hd5")

In [363]:
polyaGliomaBrainExpression = compendium[list(polyaGliomaBrainSet['th_sampleid'])]


In [416]:
basepath="/data/archive/downstream/"
postidpath="/secondary/ucsc_cgl-rnaseq-cgl-pipeline-0.0.0-0000000/RSEM/Hugo/"
filename="rsem_genes.hugo.results"
hugoLogTpmPlusOneFilePathList = glob.glob(basepath + "THR21*" + postidpath + filename)

riboGliomablastomaExpression = pd.DataFrame()
curDf = pd.read_csv(hugoLogTpmPlusOneFilePathList[0],sep='\t')
riboGliomablastomaExpression['gene']=curDf['gene_name']

for filepath in hugoLogTpmPlusOneFilePathList:
    curDf = pd.read_csv(filepath,sep='\t')
    riboGliomablastomaExpression[filepath[25:24+15]]=curDf['TPM']


In [445]:
thid = ['gene']
thid.extend(list(ribodGlioblastomaSet['th_sampleid']))
riboGliomablastomaExpression[thid].head()

,gene,THR21_0520_S01,THR21_0521_S01,THR21_0522_S01,THR21_0523_S01,THR21_0524_S01,THR21_0525_S01,THR21_0526_S01,THR21_0527_S01,THR21_0528_S01,...,THR21_0552_S01,THR21_0553_S01,THR21_0554_S01,THR21_0554_S02,THR21_0555_S01,THR21_0556_S01,THR21_0557_S01,THR21_0558_S01,THR21_0559_S01,THR21_0560_S01
0,TSPAN6,4.50,6.97,6.13,4.10,6.72,6.09,7.88,6.49,4.62,...,3.58,1.16,4.74,13.02,6.21,4.61,4.82,2.67,35.65,28.19
1,TNMD,0.08,0.10,0.03,0.06,0.14,0.27,0.03,0.03,0.01,...,0.02,0.00,0.02,0.06,0.02,0.02,0.04,0.00,0.16,0.01
2,DPM1,5.13,5.66,3.21,4.74,3.34,3.09,3.97,3.89,3.32,...,4.71,3.61,3.79,9.74,3.40,3.23,3.16,2.88,66.10,37.59
3,SCYL3,1.20,2.76,1.20,1.95,1.23,0.90,1.32,1.15,0.66,...,0.79,0.61,0.80,1.46,1.07,1.41,0.94,0.73,9.18,3.92
4,C1orf112,1.97,2.30,1.04,2.70,0.97,1.76,2.28,1.26,0.81,...,1.93,0.81,1.01,2.49,1.18,1.88,1.34,0.65,9.78,14.80


In [157]:


for filepath in listOfPathsToUMEND :
    currentSampleID = filepath[filepath.find('TH0'):filepath.find('TH0')+13]
    umendTable = pd.read_csv(filepath,sep='\t')
    if currentSampleID in listOfPathsToDisease:
        indexOfPath = index_containing_substring(listOfPathsToDisease,currentSampleID)
        f = open(listOfPathsToDisease[indexOfPath],'r')
        disease = f.read()
        f.close()
        umendTable['disease'] = np.repeat(disease,len(umendTable))
        totalUmendDisease = pd.concat([umendTable,totalUmendDisease])

KeyboardInterrupt: 